A notebook to test Pytorch.

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
#%config InlineBackend.figure_format = 'svg'
#%config InlineBackend.figure_format = 'pdf'

import numpy as np
import os
import torch

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

# font options
font = {
    #'family' : 'normal',
    #'weight' : 'bold',
    'size'   : 24
}

plt.rc('font', **font)
plt.rc('lines', linewidth=2)
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

In [ ]:
torch.zeros(3, device=torch.device('cpu'))

In [ ]:
w = torch.tensor([3,5,4.0])
w.dot(torch.ones(3))

## Torch 2d to 1d and vice versa

In [ ]:
torch.eye(3).matmul(torch.tensor([[2., 3, 1]]).T)

In [ ]:
torch.tensor([[2., 3, 1]]).T

In [ ]:
V = torch.randn(3, 1)
V

In [ ]:
V.reshape(-1)

In [ ]:
v = torch.rand(4)
v

In [ ]:
v - 1

In [ ]:
v.reshape(4, 1)

## Torch distributions
https://pytorch.org/docs/stable/distributions.html

In [ ]:
import torch.distributions as dists

In [ ]:
dis = dists.Normal(0, 1)
dis.log_prob(torch.ones(5))

### Grad of log (Normal density)

$$ \nabla_x \log p(x) = -x$$ where $p(x) = \mathcal{N}(0,1)$.

Input is a scalar

In [ ]:
x = torch.tensor([3.0], requires_grad=True)
y = 2*x
lp = dis.log_prob(y)

In [ ]:
lp.backward(retain_graph=True) 
x.grad

In [ ]:
torch.autograd.grad([lp], [x, y], retain_graph=True, only_inputs=True)

Input is n x 1

In [ ]:
X = torch.tensor([[-3, 1.0, 2]], requires_grad=True).T
Lp = dis.log_prob(X)
sum_lp = torch.sum(Lp)

In [ ]:
torch.autograd.grad(sum_lp, X, retain_graph=True, only_inputs=True)

## Multivariate normal

Assume $p(x) = \mathcal{N}(x \mid \mu, I)$. Then

$$ \nabla_x \log p(x) = -(x-\mu) $$

In [ ]:
mdist = dists.MultivariateNormal(
    loc=torch.tensor([1, 2]), covariance_matrix=torch.eye(2))

X = torch.tensor([
    [0, 0.0],
    [0, 1.0],
    [1.0, 4],
])
X.requires_grad = True
logp = mdist.log_prob(X)
torch.autograd.grad(torch.sum(logp), X, retain_graph=True, only_inputs=True)[0]

## Univariate normal

In [ ]:
x = torch.randn(5, 1)
stdnorm = dists.Normal(0,1)
stdnorm.log_prob(x)

In [ ]:
var = 2.8
stdev = var**0.5
norm = dists.Normal(0, scale=stdev)
norm.log_prob(x)

In [ ]:
stdnorm.log_prob(x/stdev) - torch.log(torch.tensor(stdev))

In [ ]:
x.shape == [5]

In [ ]:
torch.all(x <= 0).item()

## Uniform distribution

In [ ]:
U = dists.Uniform(low=-3.0, high=3.0)


## Broadcasting

In [ ]:
a = torch.arange(1, 4).reshape(3, 1)
a

In [ ]:
b = torch.arange(3, 7).reshape(1, 4)
b

In [ ]:
a+b

In [ ]:
a + b.reshape(-1)

In [ ]:
torch.exp

In [ ]:
torch.zeros((3,4))

Subtract each row

In [ ]:
A = torch.arange(18).reshape(6, 3)
A

In [ ]:
m = torch.tensor([1, 2, 3.0])
A -m 

## Remove diagonal

In [ ]:
A = torch.arange(16).reshape(4,4)
A

In [ ]:
diagA = torch.diagflat(torch.diag(A))
diagA

In [ ]:
A - diagA

## Multinomial

In [ ]:
d = 5
m = 20
mult = dists.multinomial.Multinomial(total_count=d, probs=torch.ones(d)/d)
mult.sample()

## Quadratic form
   

In [ ]:
w = torch.rand(5)
A = torch.randn(5,5)

In [ ]:
A.matmul(w).matmul(w)

## Bool to float

In [ ]:
v = torch.randn(5)
v

In [ ]:
I = v >  0
I

In [ ]:
I.to(torch.float)

## Torch scalar to normal float

In [ ]:
torch_num = torch.tensor(3.78)
torch_num.item()

## Elementwise product

In [ ]:
A = torch.randn(5,2)
B = torch.rand(5, 2)
torch.sum(A*B, 1)

## Meshgrid


In [ ]:
a = torch.arange(5.0)
print(a)
b = torch.rand(4)
print(b)

In [ ]:
torch.meshgrid(a,b)

## Clamp values in a tensor


In [ ]:
a = torch.tensor([4.0, -2.3, 2])
a

In [ ]:
a.clamp_(min=0)

## Indexing

In [ ]:
A = torch.randn(5,3)
A

## Mean and std of data

In [ ]:
X = torch.randn(8, 3)
X

In [ ]:
torch.min(X)

In [ ]:
torch.max(torch.std(X, dim=0))